In [1]:
import os
import pathlib

# # Clone the tensorflow models repository if it doesn't already exist
# if "models" in pathlib.Path.cwd().parts:
#   while "models" in pathlib.Path.cwd().parts:
#     os.chdir('..')
# elif not pathlib.Path('models').exists():
#   !git clone --depth 1 https://github.com/tensorflow/models

In [2]:
# Install the Object Detection API
# %%bash
#Change path
%cd /tf/models/research/

!protoc object_detection/protos/*.proto --python_out=.
%cp object_detection/packages/tf2/setup.py .
!python3 -m pip install .

/tf/models/research
Defaulting to user installation because normal site-packages is not writeable
Processing /tf/models/research
  Preparing metadata (setup.py) ... done
     |████████████████████████████████| 2.2 MB 37.5 MB/s            
     |████████████████████████████████| 4.9 MB 93.6 MB/s            
     |████████████████████████████████| 90 kB 16.1 MB/s            
     |████████████████████▊           | 322.6 MB 124.7 MB/s eta 0:00:02

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



     |████████████████████████████▎   | 439.9 MB 124.9 MB/s eta 0:00:01

     |████████████████████████████▋   | 445.6 MB 124.9 MB/s eta 0:00:01

     |█████████████████████████████   | 451.4 MB 124.9 MB/s eta 0:00:01

...

     |███████████████████████████████▌| 490.4 MB 100.4 MB/s eta 0:00:01

     |████████████████████████████████| 496.4 MB 100.4 MB/s eta 0:00:01

     |████████████████████████████████| 497.6 MB 100.4 MB/s eta 0:00:01

     |████████████████████████████████| 497.6 MB 1.4 kB/s              
     |████████████████████████████████| 5.8 MB 57.3 MB/s            
     |████████████████████████████████| 1.4 MB 97.2 MB/s            
     |████████████████████████████████| 462 kB 83.4 MB/s            


  Created wheel for object-detection: filename=object_detection-0.1-py3-none-any.whl size=1684296 sha256=a8282ea21fdf4061c68e247953135c37a96b1a793f63663f2546ced0fe6c1842
  Stored in directory: /tmp/pip-ephem-wheel-cache-3donspb2/wheels/89/ce/05/b0687c7bd22751a5e5a780a546434b4463698cb899345317b9
Successfully built object-detection
  Attempting uninstall: tf-models-official
    Found existing installation: tf-models-official 2.4.0
    Uninstalling tf-models-official-2.4.0:
      Successfully uninstalled tf-models-official-2.4.0
  Attempting uninstall: object-detection
    Found existing installation: object-detection 0.1
    Uninstalling object-detection-0.1:
      Successfully uninstalled object-detection-0.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow-gpu 2.7.0 requires keras<2.8,>=2.7.0rc0, but you have keras 2.8.0 which is incompatible.


In [3]:
import matplotlib
import matplotlib.pyplot as plt

import os
import random
import io
import imageio
import glob
import scipy.misc
import numpy as np
from six import BytesIO
from PIL import Image, ImageDraw, ImageFont
from IPython.display import display, Javascript
from IPython.display import Image as IPyImage

import tensorflow as tf

from object_detection.utils import label_map_util
from object_detection.utils import config_util
from object_detection.utils import visualization_utils as viz_utils
#from object_detection.utils import colab_utils
from object_detection.builders import model_builder

%matplotlib inline

Matplotlib is building the font cache; this may take a moment.


In [4]:
#run model builder test
!python3 /tf/models/research/object_detection/builders/model_builder_tf2_test.py

2022-03-03 13:19:23.202906: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-03-03 13:19:23.210008: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-03-03 13:19:23.210627: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
Running tests under Python 3.8.5: /usr/bin/python3
[ RUN      ] ModelBuilderTF2Test.test_create_center_net_deepmac
2022-03-03 13:19:23.218273: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critic

INFO:tensorflow:time(__main__.ModelBuilderTF2Test.test_create_faster_rcnn_models_from_config_mask_rcnn_without_matmul): 0.09s
I0303 13:19:28.727180 139732260468544 test_util.py:2373] time(__main__.ModelBuilderTF2Test.test_create_faster_rcnn_models_from_config_mask_rcnn_without_matmul): 0.09s
[       OK ] ModelBuilderTF2Test.test_create_faster_rcnn_models_from_config_mask_rcnn_without_matmul
[ RUN      ] ModelBuilderTF2Test.test_create_rfcn_model_from_config
INFO:tensorflow:time(__main__.ModelBuilderTF2Test.test_create_rfcn_model_from_config): 0.09s
I0303 13:19:28.814375 139732260468544 test_util.py:2373] time(__main__.ModelBuilderTF2Test.test_create_rfcn_model_from_config): 0.09s
[       OK ] ModelBuilderTF2Test.test_create_rfcn_model_from_config
[ RUN      ] ModelBuilderTF2Test.test_create_ssd_fpn_model_from_config
INFO:tensorflow:time(__main__.ModelBuilderTF2Test.test_create_ssd_fpn_model_from_config): 0.17s
I0303 13:19:28.985078 139732260468544 test_util.py:2373] time(__main__.Model

I0303 13:19:32.108928 139732260468544 ssd_efficientnet_bifpn_feature_extractor.py:142] EfficientDet EfficientNet backbone version: efficientnet-b2
I0303 13:19:32.109083 139732260468544 ssd_efficientnet_bifpn_feature_extractor.py:144] EfficientDet BiFPN num filters: 112
I0303 13:19:32.109141 139732260468544 ssd_efficientnet_bifpn_feature_extractor.py:145] EfficientDet BiFPN num iterations: 5
I0303 13:19:32.110720 139732260468544 efficientnet_model.py:147] round_filter input=32 output=32
I0303 13:19:32.125168 139732260468544 efficientnet_model.py:147] round_filter input=32 output=32
I0303 13:19:32.125270 139732260468544 efficientnet_model.py:147] round_filter input=16 output=16
I0303 13:19:32.233585 139732260468544 efficientnet_model.py:147] round_filter input=16 output=16
I0303 13:19:32.233736 139732260468544 efficientnet_model.py:147] round_filter input=24 output=24
I0303 13:19:32.439237 139732260468544 efficientnet_model.py:147] round_filter input=24 output=24
I0303 13:19:32.439392 13

I0303 13:19:35.730765 139732260468544 efficientnet_model.py:147] round_filter input=16 output=24
I0303 13:19:35.730921 139732260468544 efficientnet_model.py:147] round_filter input=24 output=32
I0303 13:19:36.001180 139732260468544 efficientnet_model.py:147] round_filter input=24 output=32
I0303 13:19:36.001344 139732260468544 efficientnet_model.py:147] round_filter input=40 output=56
I0303 13:19:36.508440 139732260468544 efficientnet_model.py:147] round_filter input=40 output=56
I0303 13:19:36.508596 139732260468544 efficientnet_model.py:147] round_filter input=80 output=112
I0303 13:19:36.925839 139732260468544 efficientnet_model.py:147] round_filter input=80 output=112
I0303 13:19:36.926000 139732260468544 efficientnet_model.py:147] round_filter input=112 output=160
I0303 13:19:37.345850 139732260468544 efficientnet_model.py:147] round_filter input=112 output=160
I0303 13:19:37.346003 139732260468544 efficientnet_model.py:147] round_filter input=192 output=272
I0303 13:19:37.894185 

I0303 13:19:43.270913 139732260468544 efficientnet_model.py:147] round_filter input=112 output=200
I0303 13:19:43.271083 139732260468544 efficientnet_model.py:147] round_filter input=192 output=344
I0303 13:19:44.024026 139732260468544 efficientnet_model.py:147] round_filter input=192 output=344
I0303 13:19:44.024190 139732260468544 efficientnet_model.py:147] round_filter input=320 output=576
I0303 13:19:44.229383 139732260468544 efficientnet_model.py:147] round_filter input=1280 output=2304
I0303 13:19:44.254624 139732260468544 efficientnet_model.py:457] Building model efficientnet with params ModelConfig(width_coefficient=1.8, depth_coefficient=2.6, resolution=528, dropout_rate=0.5, blocks=(BlockConfig(input_filters=32, output_filters=16, kernel_size=3, num_repeat=1, expand_ratio=1, strides=(1, 1), se_ratio=0.25, id_skip=True, fused_conv=False, conv_type='depthwise'), BlockConfig(input_filters=16, output_filters=24, kernel_size=3, num_repeat=2, expand_ratio=6, strides=(2, 2), se_rati

In [5]:
def load_image_into_numpy_array(path):
  """Load an image from file into a numpy array.

  Puts image into numpy array to feed into tensorflow graph.
  Note that by convention we put it into a numpy array with shape
  (height, width, channels), where channels=3 for RGB.

  Args:
    path: a file path.

  Returns:
    uint8 numpy array with shape (img_height, img_width, 3)
  """
  img_data = tf.io.gfile.GFile(path, 'rb').read()
  image = Image.open(BytesIO(img_data))
  (im_width, im_height) = image.size
  return np.array(image.getdata()).reshape(
      (im_height, im_width, 3)).astype(np.uint8)

def plot_detections(image_np,
                    boxes,
                    classes,
                    scores,
                    category_index,
                    figsize=(12, 16),
                    image_name=None):
  """Wrapper function to visualize detections.

  Args:
    image_np: uint8 numpy array with shape (img_height, img_width, 3)
    boxes: a numpy array of shape [N, 4]
    classes: a numpy array of shape [N]. Note that class indices are 1-based,
      and match the keys in the label map.
    scores: a numpy array of shape [N] or None.  If scores=None, then
      this function assumes that the boxes to be plotted are groundtruth
      boxes and plot all boxes as black with no classes or scores.
    category_index: a dict containing category dictionaries (each holding
      category index `id` and category name `name`) keyed by category indices.
    figsize: size for the figure.
    image_name: a name for the image file.
  """
  image_np_with_annotations = image_np.copy()
  viz_utils.visualize_boxes_and_labels_on_image_array(
      image_np_with_annotations,
      boxes,
      classes,
      scores,
      category_index,
      use_normalized_coordinates=True,
      min_score_thresh=0.8)
  if image_name:
    plt.imsave(image_name, image_np_with_annotations)
  else:
    plt.imshow(image_np_with_annotations)

In [6]:
#Change
train_record_fname = "/tf/carpartdetectiondataset/train/cars.tfrecord"
validation_record_fname = "/tf/carpartdetectiondataset/train/cars.tfrecord"
test_record_fname = "/tf/carpartdetectiondataset/test/cars.tfrecord"


In [7]:
label_map_pbtxt_fname = "/tf/carpartdetectiondataset/train/cars_label_map.pbtxt"

In [8]:
##change chosen model to deploy different models available in the TF2 object detection zoo
MODELS_CONFIG = {
    'efficientdet-d0': {
        'model_name': 'efficientdet_d0_coco17_tpu-32',
        'base_pipeline_file': 'ssd_efficientdet_d0_512x512_coco17_tpu-8.config',
        'pretrained_checkpoint': 'efficientdet_d0_coco17_tpu-32.tar.gz',
        'batch_size': 4
    },
    'efficientdet-d1': {
        'model_name': 'efficientdet_d1_coco17_tpu-32',
        'base_pipeline_file': 'ssd_efficientdet_d1_640x640_coco17_tpu-8.config',
        'pretrained_checkpoint': 'efficientdet_d1_coco17_tpu-32.tar.gz',
        'batch_size': 8
    },
    'efficientdet-d2': {
        'model_name': 'efficientdet_d2_coco17_tpu-32',
        'base_pipeline_file': 'ssd_efficientdet_d2_768x768_coco17_tpu-8.config',
        'pretrained_checkpoint': 'efficientdet_d2_coco17_tpu-32.tar.gz',
        'batch_size': 16
    },
        'efficientdet-d3': {
        'model_name': 'efficientdet_d3_coco17_tpu-32',
        'base_pipeline_file': 'ssd_efficientdet_d3_896x896_coco17_tpu-32.config',
        'pretrained_checkpoint': 'efficientdet_d3_coco17_tpu-32.tar.gz',
        'batch_size': 8
    }
}

In [18]:
chosen_model = 'efficientdet-d1'
classes = 18
num_steps = 875
num_eval_steps = 100 #Perform evaluation after so many steps

#numofsteps/numoftrainingdata * batch size
#Currently we have 350, 

model_name = MODELS_CONFIG[chosen_model]['model_name']
pretrained_checkpoint = MODELS_CONFIG[chosen_model]['pretrained_checkpoint']
base_pipeline_file = MODELS_CONFIG[chosen_model]['base_pipeline_file']
batch_size = MODELS_CONFIG[chosen_model]['batch_size'] #if you can fit a large batch in memory, it may speed up your training

In [10]:
#download pretrained weights

#Change
%mkdir /tf/models/research/deploy/
%cd /tf/models/research/deploy/
#
import tarfile
download_tar = 'http://download.tensorflow.org/models/object_detection/tf2/20200711/' + pretrained_checkpoint

!wget {download_tar}
tar = tarfile.open(pretrained_checkpoint)
tar.extractall()
tar.close()

/tf/models/research/deploy
--2022-03-03 13:19:58--  http://download.tensorflow.org/models/object_detection/tf2/20200711/efficientdet_d1_coco17_tpu-32.tar.gz
Resolving download.tensorflow.org (download.tensorflow.org)... 142.250.184.208, 2a00:1450:4001:830::2010
Connecting to download.tensorflow.org (download.tensorflow.org)|142.250.184.208|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 51839363 (49M) [application/x-tar]
Saving to: ‘efficientdet_d1_coco17_tpu-32.tar.gz’

100%[======================================>] 51,839,363   103MB/s   in 0.5s   

2022-03-03 13:19:59 (103 MB/s) - ‘efficientdet_d1_coco17_tpu-32.tar.gz’ saved [51839363/51839363]



In [11]:
#Change
%cd /tf/models/research/deploy
download_config = 'https://raw.githubusercontent.com/tensorflow/models/master/research/object_detection/configs/tf2/' + base_pipeline_file
!wget {download_config}

/tf/models/research/deploy
--2022-03-03 13:19:59--  https://raw.githubusercontent.com/tensorflow/models/master/research/object_detection/configs/tf2/ssd_efficientdet_d1_640x640_coco17_tpu-8.config
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.110.133, 185.199.111.133, 185.199.108.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.110.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 4630 (4.5K) [text/plain]
Saving to: ‘ssd_efficientdet_d1_640x640_coco17_tpu-8.config’

100%[======================================>] 4,630       --.-K/s   in 0s      

2022-03-03 13:19:59 (72.1 MB/s) - ‘ssd_efficientdet_d1_640x640_coco17_tpu-8.config’ saved [4630/4630]



In [19]:
#prepare
pipeline_fname = '/tf/models/research/deploy/' + base_pipeline_file
fine_tune_checkpoint = '/tf/models/research/deploy/' + model_name + '/checkpoint/ckpt-0'

def get_num_classes(pbtxt_fname):
    from object_detection.utils import label_map_util
    label_map = label_map_util.load_labelmap(pbtxt_fname)
    categories = label_map_util.convert_label_map_to_categories(
        label_map, max_num_classes=90, use_display_name=True)
    category_index = label_map_util.create_category_index(categories)
    return len(category_index.keys())
num_classes = get_num_classes(label_map_pbtxt_fname)

In [20]:
import re

#Change#
%cd /tf/models/research/deploy
print('writing custom configuration file')

with open(pipeline_fname) as f:
    s = f.read()

/tf/models/research/deploy
writing custom configuration file


In [21]:
with open('pipeline_file.config', 'w') as f:
    
    # fine_tune_checkpoint
    s = re.sub('fine_tune_checkpoint: ".*?"',
               'fine_tune_checkpoint: "{}"'.format(fine_tune_checkpoint), s)
    
    # tfrecord files train and test.
    s = re.sub(
        '(input_path: ".*?)(PATH_TO_BE_CONFIGURED/train)(.*?")', 'input_path: "{}"'.format(train_record_fname), s)
    s = re.sub(
        '(input_path: ".*?)(PATH_TO_BE_CONFIGURED/val)(.*?")', 'input_path: "{}"'.format(test_record_fname), s)

    # label_map_path
    s = re.sub(
        'label_map_path: ".*?"', 'label_map_path: "{}"'.format(label_map_pbtxt_fname), s)

    # Set training batch_size.
    s = re.sub('batch_size: [0-9]+',
               'batch_size: {}'.format(batch_size), s)

    # Set training steps, num_steps
    s = re.sub('num_steps: [0-9]+',
               'num_steps: {}'.format(num_steps), s)
    
    # Set number of classes num_classes.
    s = re.sub('num_classes: [0-9]+',
               'num_classes: {}'.format(num_classes), s)
    
    #fine-tune checkpoint type
    s = re.sub(
        'fine_tune_checkpoint_type: "classification"', 'fine_tune_checkpoint_type: "{}"'.format('detection'), s)
        
    f.write(s)

In [22]:
#Change
pipeline_file = '/tf/models/research/deploy/pipeline_file.config'
model_dir = '/tf/training/'

In [23]:
#Change
!python3 /tf/models/research/object_detection/model_main_tf2.py \
    --pipeline_config_path={pipeline_file} \
    --model_dir={model_dir} \
    --alsologtostderr \
    --num_train_steps={875} \
    --sample_1_of_n_eval_examples=1 \
    --num_eval_steps={num_eval_steps}

/home/ec2-user/.local/lib/python3.8/site-packages/tensorflow_addons/utils/ensure_tf_install.py:53: UserWarning: Tensorflow Addons supports using Python ops for all Tensorflow versions above or equal to 2.5.0 and strictly below 2.8.0 (nightly versions are not supported). 
 The versions of TensorFlow you are currently using is 2.8.0 and is not supported. 
Some things might work, some things might not.
If you were to encounter a bug, do not file an issue.
If you want to make sure you're using a tested and supported configuration, either change the TensorFlow version or the TensorFlow Addons's version. 
You can find the compatibility matrix in TensorFlow Addon's readme:
https://github.com/tensorflow/addons
  warnings.warn(
2022-03-03 13:31:27.327606: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-03-03 13:31:27.334692: I tensorflow/stream_exe

I0303 13:31:29.094492 139754436839232 efficientnet_model.py:147] round_filter input=40 output=40
I0303 13:31:29.094658 139754436839232 efficientnet_model.py:147] round_filter input=80 output=80
I0303 13:31:29.530976 139754436839232 efficientnet_model.py:147] round_filter input=80 output=80
I0303 13:31:29.531136 139754436839232 efficientnet_model.py:147] round_filter input=112 output=112
I0303 13:31:29.974277 139754436839232 efficientnet_model.py:147] round_filter input=112 output=112
I0303 13:31:29.974433 139754436839232 efficientnet_model.py:147] round_filter input=192 output=192
I0303 13:31:30.709615 139754436839232 efficientnet_model.py:147] round_filter input=192 output=192
I0303 13:31:30.709812 139754436839232 efficientnet_model.py:147] round_filter input=320 output=320
I0303 13:31:30.949569 139754436839232 efficientnet_model.py:147] round_filter input=1280 output=1280
I0303 13:31:30.992386 139754436839232 efficientnet_model.py:457] Building model efficientnet with params ModelCon

W0303 13:32:32.603495 139733909628672 utils.py:76] Gradients do not exist for variables ['stack_6/block_1/expand_bn/gamma:0', 'stack_6/block_1/expand_bn/beta:0', 'stack_6/block_1/depthwise_conv2d/depthwise_kernel:0', 'stack_6/block_1/depthwise_bn/gamma:0', 'stack_6/block_1/depthwise_bn/beta:0', 'stack_6/block_1/project_bn/gamma:0', 'stack_6/block_1/project_bn/beta:0', 'top_bn/gamma:0', 'top_bn/beta:0'] when minimizing the loss. If you're using `model.compile()`, did you forget to provide a `loss`argument?
W0303 13:32:45.901269 139733909628672 utils.py:76] Gradients do not exist for variables ['stack_6/block_1/expand_bn/gamma:0', 'stack_6/block_1/expand_bn/beta:0', 'stack_6/block_1/depthwise_conv2d/depthwise_kernel:0', 'stack_6/block_1/depthwise_bn/gamma:0', 'stack_6/block_1/depthwise_bn/beta:0', 'stack_6/block_1/project_bn/gamma:0', 'stack_6/block_1/project_bn/beta:0', 'top_bn/gamma:0', 'top_bn/beta:0'] when minimizing the loss. If you're using `model.compile()`, did you forget to prov

INFO:tensorflow:Step 900 per-step time 1.787s
I0303 14:00:26.823734 139754436839232 model_lib_v2.py:705] Step 900 per-step time 1.787s
INFO:tensorflow:{'Loss/classification_loss': 0.40071613,
 'Loss/localization_loss': 0.1433155,
 'Loss/regularization_loss': 0.030044263,
 'Loss/total_loss': 0.5740759,
 'learning_rate': 0.02944}
I0303 14:00:26.824039 139754436839232 model_lib_v2.py:708] {'Loss/classification_loss': 0.40071613,
 'Loss/localization_loss': 0.1433155,
 'Loss/regularization_loss': 0.030044263,
 'Loss/total_loss': 0.5740759,
 'learning_rate': 0.02944}


In [26]:
#Change
#%load_ext tensorboard
!tensorboard --logdir '/tf/training/train'

2022-03-03 14:07:45.468787: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-03-03 14:07:45.475769: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-03-03 14:07:45.476428: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero

NOTE: Using experimental fast data loading logic. To disable, pass
    "--load_fast=false" and report issues on GitHub. More details:
    https://github.com/tensorflow/tensorboard/issues/4784

Serving TensorBoard on localhost; to expose to the network, use a proxy or pass --bind_all
TensorBoard 2.8.0 at http://localhost:6007/ (Press CTRL

In [ ]:
#Exporting
#run conversion script
import re
import numpy as np

output_directory = '/tf/fine_tuned_model'

#place the model weights you would like to export here
last_model_path = '/tf/training/'
print(last_model_path)
!python3 /tf/models/research/object_detection/exporter_main_v2.py \
    --trained_checkpoint_dir {last_model_path} \
    --output_directory {output_directory} \
    --pipeline_config_path {pipeline_file}

/tf/training/
2022-03-03 14:24:03.310629: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-03-03 14:24:03.317567: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-03-03 14:24:03.318216: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:936] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-03-03 14:24:03.326408: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appr